captured data set will be pre-processed in three diﬀerent ways <br>
at the end of the pre-processing, you will have three data sets: PS1, PS2, and PS3. <br>
PS1 will contain packets, PS2 will contain ﬂows, and PS3 will only contain TCP connections. <br>
3 datasets will be analysed separately in the data analysis phase <br>

In [32]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import csv
import statsmodels.api as sm
import datetime
import statistics
import statsmodels.api as sm
import pylab
from distfit import distfit

# Pre-processing
Wireshark->oneday.pcap
## PS1
oneday.pcap->Wireshark-export packet dissections as CSV->PS1.csv
## PS2
oneday.pcap-> crl_flow -> PS2.t2 -> t2_to_csv() -> PS2.csv
## PS3
oneday.pcap-> tcptrace -> oneday-tcp.csv

In [34]:
# for PS2 
def t2_to_csv(input_file, output):
    # By observing the file, we can find that blank lines and lines with "#" in front of them can be removed to get the clean data.
    clean = []
    with open(input_file , 'r') as f1:
        for line in f1:
            line = line.strip()
            if line.startswith("#") or len(line) == 0:
                pass
            else:
                x = line.split('\t')
                clean.append(x)

    # From list to csv
    header = ['src', 'dst', 'pro', 'ok', 'sport', 'dport', 'pkts', 'bytes',
    'flows', 'first', 'latest']

    with open(output, 'w', newline='') as f2:
        out = csv.writer(f2)
        out.writerow(header)
        out.writerows(clean)

t2_to_csv('./t1_data/PS2.t2','./t1_data/PS2.csv')

# data analysis
Each plot should contain a short description and also descriptive labels for the axis.

### Packet data PS1
1.1: Visualise packet distribution by port numbers.

In [ ]:
# hist
df_ps1 = pd.read_csv('./t1_data/PS1.csv')
extract all port information to csv
src_ports = []
dst_ports = []
for index, row in df_ps1.iterrows():
    info = row['Info']
    if '  >  ' in info:
        dport = info.split('  >  ')[1].split(' ')[0]
        dst_ports.append(int(dport))
        sport = info.split('  >  ')[0].split(' ')[-1]
        src_ports.append(int(sport))
src_ports = pd.Series(src_ports)
dst_ports = pd.Series(dst_ports)
src_ports.to_csv("./t1_data/1_1_src.csv",index=False)
dst_ports.to_csv("./t1_data/1_1_dst.csv",index=False)

src_ports = pd.read_csv("./t1_data/1_1_src.csv")
dst_ports = pd.read_csv("./t1_data/1_1_dst.csv")
src_ports = src_ports.iloc[:,0]
dst_ports = dst_ports.iloc[:,0]
src_port_count = src_ports.unique().shape[0]
dst_port_count = dst_ports.unique().shape[0]

plt.figure(figsize=(20,15))
src_ports.plot.hist(bins = src_port_count)
plt.title("packet distribtion by source port")
plt.yscale('log')
plt.xlabel("port number")
plt.ylabel("packets(log)")

plt.figure(figsize=(20,15))
dst_ports.plot.hist(bins = dst_port_count)
plt.title("packet distribtion by dst port")
plt.yscale('log')
plt.xlabel("port number")
plt.ylabel("packets(log)")


In [ ]:
# 10 most common ports box plot
src_ports = pd.read_csv("./t1_data/1_1_src.csv")
dst_ports = pd.read_csv("./t1_data/1_1_dst.csv")
src_ports = src_ports.iloc[:,0]
dst_ports = dst_ports.iloc[:,0]

# src
ports = src_ports.value_counts().index.tolist()
freqs = src_ports.value_counts().values.tolist()

N_most_common = 10
mc_ports = [str(x) for x in ports[0:N_most_common]]
mc_freqs = freqs[0:N_most_common]

plt.figure(figsize=(10,8))
plt.bar(mc_ports, mc_freqs)
plt.title(str(N_most_common)+" most common src ports")
plt.xlabel("port number")
plt.ylabel("frequency")

# dst
ports = dst_ports.value_counts().index.tolist()
freqs = dst_ports.value_counts().values.tolist()

N_most_common = 10
mc_ports = [str(x) for x in ports[0:N_most_common]]
mc_freqs = freqs[0:N_most_common]

plt.figure(figsize=(10,8))
plt.bar(mc_ports, mc_freqs)
plt.title(str(N_most_common)+" most common dst ports")
plt.xlabel("port number")
plt.ylabel("frequency")

1.2: Plot traﬃc volume as a function of time with at least two suﬃciently diﬀerent time scales.

In [ ]:
df_ps1 = pd.read_csv('./t1_data/PS1.csv')
# two different time scales(in seconds)
time_scale_1 = 60
time_scale_2 = 300

volumes_list = []
cur_period = 0
period_i = 0
for index, row in df_ps1.iterrows():
    time = float(row['Time'])
    if time>=period_i*time_scale_1 and time<=(period_i+1)*time_scale_1:
        cur_period += int(row['Length'])
    else:
        volumes_list.append(cur_period)
        cur_period = 0
        period_i += 1
plt.figure(figsize=(10,8))
plt.plot(np.array(volumes_list))
plt.title("traffic volume time plot in bytes, per "+str(time_scale_1)+" seconds")
plt.xlabel("time")
plt.ylabel("Traffic Volume[bytes]")

volumes_list = []
cur_period = 0
period_i = 0
for index, row in df_ps1.iterrows():
    time = float(row['Time'])
    if time>=period_i*time_scale_2 and time<=(period_i+1)*time_scale_2:
        cur_period += int(row['Length'])
    else:
        volumes_list.append(cur_period)
        cur_period = 0
        period_i += 1 
plt.figure(figsize=(10,8))
plt.plot(np.array(volumes_list))
plt.title("traffic volume time plot in bytes, per "+str(time_scale_2)+" seconds")
plt.xlabel("time")
plt.ylabel("Traffic Volume[bytes]")

1.3: Plot packet length distribution (use bins of width 1 byte), its empirical cumulative distribution function and key summary statistics.

In [ ]:
df_ps1 = pd.read_csv('./t1_data/PS1.csv')
lengths = df_ps1.loc[:,'Length']

# histogram
plt.figure(figsize=(16,9))
lengths.plot.hist(bins=lengths.unique().shape[0])
plt.title("Packet length distribtion histogram")
plt.xlabel("Packet length in bytes")
plt.ylabel("Frequency")

# key statistics
print("key summary statistics")
print("Maximum: "+str(lengths.max()))
print("Minimum: "+str(lengths.min()))
print("Mean: "+str(lengths.mean()))
print("Median: "+str(lengths.median()))
print("Standard deviation: "+str(lengths.std()))

# Empirical CDF
lengths = np.array(lengths)
ecdf = sm.distributions.ECDF(lengths)
x = np.linspace(0,max(lengths))
y1 = ecdf(x)

types = 'linear'
plt.figure(figsize=(10,8))
plt.plot(x, y1, linewidth = '1')
plt.xlabel('Variable')
plt.ylabel('CDF')
plt.yscale(types)
plt.title(f'Empirical CDF of length distribution using {types} values')

types = 'log'
plt.figure(figsize=(10,8))
plt.plot(x, y1, linewidth = '1')
plt.xlabel('Variable')
plt.ylabel('CDF')
plt.yscale(types)
plt.title(f'Empirical CDF of length distribution using {types} values')

### Flow data PS2
• 1.4: Visualise ﬂow distribution by port. <br>
• 1.5: Plot traﬃc volume as a function of time with at least two suﬃciently diﬀerent time scales. <br>
• 1.6: Visualise ﬂow distribution by country. Hint: use GeoIP to transform IP addresses to countries. If you have anonymised IP addresses, the results can be misleading (depending on level of anonymisation). <br>
• 1.7: Plot origin-destination pairs both by data volume and by ﬂows (Zipf type plot). <br>
• 1.8: Plot ﬂow length distribution, its empirical cumulative distribution function and key summary statistics. <br>
• 1.9: Fit a distribution for the ﬂow lengths and validate the model. <br>
• 1.10: Compare the number of ﬂows with 1, 10, 60, 120 and 1800 second timeouts. In this, you need to generate ﬂow data multiple times. <br>

In [37]:
# read csv
df_ps2 = pd.read_csv('./t1_data/PS2.csv')
# sort the flow data by first time
df_ps2 = df_ps2.sort_values('first')

In [ ]:
# 1.4
src_ports = df_ps2.loc[:,'sport']
dst_ports = df_ps2.loc[:,'dport']
src_port_count = src_ports.unique().shape[0]
dst_port_count = dst_ports.unique().shape[0]

plt.figure(figsize=(20,15))
src_ports.plot.hist(bins = src_port_count)
plt.title("packet distribtion by source port")
plt.yscale('log')
plt.xlabel("port number")
plt.ylabel("packets(log)")

plt.figure(figsize=(20,15))
dst_ports.plot.hist(bins = dst_port_count)
plt.title("packet distribtion by dst port")
plt.yscale('log')
plt.xlabel("port number")
plt.ylabel("packets(log)")

#src
ports = src_ports.value_counts().index.tolist()
freqs = src_ports.value_counts().values.tolist()
N_most_common = 10
mc_ports = [str(x) for x in ports[0:N_most_common]]
mc_freqs = freqs[0:N_most_common]
plt.figure(figsize=(10,8))
plt.bar(mc_ports, mc_freqs)
plt.title(str(N_most_common)+" most common src ports")
plt.xlabel("port number")
plt.ylabel("frequency")
# dst
ports = dst_ports.value_counts().index.tolist()
freqs = dst_ports.value_counts().values.tolist()
N_most_common = 10
mc_ports = [str(x) for x in ports[0:N_most_common]]
mc_freqs = freqs[0:N_most_common]
plt.figure(figsize=(10,8))
plt.bar(mc_ports, mc_freqs)
plt.title(str(N_most_common)+" most common dst ports")
plt.xlabel("port number")
plt.ylabel("frequency")

In [ ]:
# 1.5
init_time = float(df_ps2.loc[:,'first'].min())
time_scale_1 = 60
time_scale_2 = 300

volumes_list = []
cur_period = 0
period_i = 0
for index, row in df_ps2.iterrows():
    time = float(row['first'])
    if time-init_time >= period_i*time_scale_1 and time-init_time <= (period_i+1)*time_scale_1:
        cur_period += int(row['bytes'])
    else:
        volumes_list.append(cur_period)
        cur_period = 0
        period_i += 1

plt.figure(figsize=(10,8))
plt.plot(np.array(volumes_list))
plt.title("traffic volume time plot in bytes, per "+str(time_scale_1)+" seconds")
plt.xlabel("time")
plt.ylabel("Traffic Volume[bytes]")

volumes_list = []
cur_period = 0
period_i = 0
for index, row in df_ps2.iterrows():
    time = float(row['first'])
    if time-init_time >= period_i*time_scale_2 and time-init_time <= (period_i+1)*time_scale_2:
        cur_period += int(row['bytes'])
    else:
        volumes_list.append(cur_period)
        cur_period = 0
        period_i += 1

plt.figure(figsize=(10,8))
plt.plot(np.array(volumes_list))
plt.title("traffic volume time plot in bytes, per "+str(time_scale_2)+" seconds")
plt.xlabel("time")
plt.ylabel("Traffic Volume[bytes]")


In [ ]:
# 1.6
src_address = df_ps2.loc[:,'src']
dst_address = df_ps2.loc[:,'dst']

from geolite2 import geolite2
reader = geolite2.reader()
country_count = dict()
not_match_list = []

for index, item in src_address.iteritems():
    match = reader.get(item)
    if match:
        if 'country' in match:
            country_code = match['country']['iso_code']
        else:
            country_code = match['continent']['code']
        if country_code in country_count:
            country_count[country_code] += 1
        else:
            country_count[country_code] = 1
    else:
        not_match_list.append(item)
# country_count['OTHER'] = len(not_match_list)
keys = country_count.keys()
values = country_count.values()
plt.figure(figsize=(10,8))
plt.bar(keys, values)
plt.title('Flow distribution by country(source)')
plt.xlabel('Country code')
plt.ylabel('flow frequency')


for index, item in dst_address.iteritems():
    match = reader.get(item)
    if match:
        if 'country' in match:
            country_code = match['country']['iso_code']
        else:
            country_code = match['continent']['code']
        if country_code in country_count:
            country_count[country_code] += 1
        else:
            country_count[country_code] = 1
    else:
        not_match_list.append(item)
# country_count['OTHER'] = len(not_match_list)
keys = country_count.keys()
values = country_count.values()
plt.figure(figsize=(10,8))
plt.bar(keys, values)
plt.title('Flow distribution by country(destination)')
plt.xlabel('Country code')
plt.ylabel('flow frequency')


In [ ]:
# 1.7
# count pairs
pairs = df_ps2.groupby(['src','dst'])
t_flows = pairs['flows'].sum().reset_index().sort_values('flows',ascending=False)
t_bytes = pairs['bytes'].sum().reset_index().sort_values('bytes',ascending=False)
#zipf plot by bytes
total_bytes = t_bytes.loc[:,'bytes'].sum()
temp = t_bytes.loc[:,'bytes'].tolist()
freqs = [item/total_bytes for item in temp]
plt.figure(figsize=(10,8))
plt.plot(np.arange(1,len(temp)+1), freqs)
plt.title('Zipf plot by bytes')
plt.xlabel('rank')
plt.ylabel('frequency')
#zipf plot by bytes
total_flows = t_flows.loc[:,'flows'].sum()
temp = t_flows.loc[:,'flows'].tolist()
freqs = [item/total_flows for item in temp]
plt.figure(figsize=(10,8))
plt.plot(np.arange(1,len(temp)+1), freqs)
plt.title('Zipf plot by flows')
plt.xlabel('rank')
plt.ylabel('frequency')


In [ ]:
# 1.8
# distribution
flow_lengths = df_ps2.loc[:,"bytes"]
plt.figure(figsize=(10,8))
flow_lengths.plot.hist(bins=100)
plt.title("flow length distribtion")
plt.xlabel("length in bytes")
plt.ylabel("frequency")
plt.yscale('linear')

# ecdf
lengths_list = np.array(flow_lengths)
ecdf = sm.distributions.ECDF(lengths_list)
max1 = flow_lengths.max()
x = np.linspace(0,max1)
y1 = ecdf(x)

types = 'linear'
plt.figure(figsize=(10,8))
plt.plot(x, y1, linewidth = '1')
plt.xlabel('Variable')
plt.ylabel('CDF')
plt.yscale(types)
plt.title(f'Empirical CDF of length distribution using {types} values')

types = 'log'
plt.figure(figsize=(10,8))
plt.plot(x, y1, linewidth = '1')
plt.xlabel('Variable')
plt.ylabel('CDF')
plt.yscale(types)
plt.title(f'Empirical CDF of length distribution using {types} values')
# key statistics
# mean, variance, max, min, median
mean = flow_lengths.mean()
median = flow_lengths.median()
max = flow_lengths.max()
min = flow_lengths.min()
std = flow_lengths.std()
var = flow_lengths.var()
key_stat = {"items":["mean","median","max","min","st dev."],
    "values":[str(x) for x in [mean,median,max,min,std]]
}
df = pd.DataFrame(key_stat)
display(df)



In [ ]:
# 1.9
# fit and validate
df_ps2 = pd.read_csv("./t1_data/PS2.csv")
flow_lengths = df_ps2.loc[:,"bytes"]
flow_lengths = flow_lengths.values
dist = distfit()
dist.fit_transform(flow_lengths)
print(dist.summary)
dist.plot_summary()

In [62]:
# 1.10
timeouts = [1, 10, 60, 120, 1800]
for timeout in timeouts:
    df = pd.read_csv("./t1_data/PS2-"+str(timeout)+".csv")
    print("timeout = "+str(timeout)+" seconds")
    print(df.shape[0])

timeout = 1 seconds
108946
timeout = 10 seconds
77588
timeout = 60 seconds
39625
timeout = 120 seconds
38284
timeout = 1800 seconds
34748


## PS3

In [3]:
data_list = []
with open("./t1_data/oneday-tcp.csv") as f:
    for line in f:
        if 'conn_#' in line:
            headers = line.split(',')
        elif '#' not in line and ',' in line:
            data_list.append(line.split(','))
df_ps3 = pd.DataFrame(data_list,columns=headers)

In [ ]:
retrans_a2b = pd.to_numeric(df_ps3.loc[:,'max_#_retrans_a2b'])
retrans_b2a = pd.to_numeric(df_ps3.loc[:,'max_#_retrans_b2a'])
rtt_a2b = pd.to_numeric(df_ps3.loc[:,'RTT_samples_a2b'])
rtt_b2a = pd.to_numeric(df_ps3.loc[:,'RTT_samples_b2a'])
x_axis = np.arange(1,df_ps3.shape[0]+1)
plt.figure(figsize=(16,9))
plt.plot(x_axis,retrans_a2b/retrans_a2b.max(), alpha=0.5)
plt.plot(x_axis,rtt_a2b/rtt_a2b.max(), alpha=0.5)
plt.legend(['retransmission_a2b','RTT_a2b'])
plt.title("association between retransmission and RTT (a to b)")
plt.xlabel("sample index")
plt.ylabel("value")

plt.figure(figsize=(16,9))
plt.plot(x_axis,retrans_b2a/retrans_b2a.max(), alpha=0.5)
plt.plot(x_axis,rtt_b2a/rtt_b2a.max(), alpha=0.5)
plt.legend(['RTT_b2a','retransmission_b2a'])
plt.title("association between retransmission and RTT (b to a)")
plt.xlabel("sample index")
plt.ylabel("value")


# variance
print("variance of retransmission(a to b):")
print(retrans_a2b.var())
print("variance of retransmission(b to a):")
print(retrans_b2a.var())
print("variance of RTT(a to b):")
print(rtt_a2b.var())
print("variance of RTT(b to a):")
print(rtt_b2a.var())

1.12

In [31]:
df_ps2 = pd.read_csv("./t1_data/PS2.csv")
df_ps2 = df_ps2.sort_values('first')
start = df_ps3.loc[:,'first_packet'].tolist()
end = df_ps3.loc[:,'last_packet'].tolist()
start_list = [float(x) for x in start]
end_list = [float(x) for x in end]


i = 0
total_volume = 0
while start_list != []:
    start2 = df_ps2.iloc[i,9]
    end2 = df_ps2.iloc[i,10]
    if start2 >= start_list[0] and end2 <= end_list[0]:
        total_volume += df_ps2.iloc[i,7]
        i+=1
    else:
        if end2 >= end_list[0]:
            start_list.pop(0)
            end_list.pop(0)
        elif end2 < end_list[0]:
            i+=1
    
print("total traffic volume during TCP connections(in bytes):")
print(total_volume)


total traffic volume during TCP connections(in bytes):
442593154
